In [1]:
from agents_experiments import (step_interact, ControlledAgent, IdleAgent)
import logging
from luxai_s2 import LuxAI_S2
import matplotlib.pyplot as plt
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [2]:
def diagnose(obs_list, rew_list, don_list, inf_list, log=logging.info):
    """
    What questions do I have?

    step
    board matrices (changed?)
    factories
    units
    resource levels
    actions
    time elapsed

    and more questions about the specifics of my algo
    ice and ore neighborhoods of each plant
    plant and resource assignments to robots
    shortest paths + weights from robots to tiles
    """
    o, r, d, i = obs_list[-1], rew_list[-1], don_list[-1], inf_list[-1]
    p0 = o['player_0']
    p1 = o['player_1']
    res = p0['real_env_steps']
    assert res == p1['real_env_steps']
    log(f"For both players real_env_steps={res}")
    num_plants0 = p0['teams']['player_0']['factories_to_place']
    num_plants1 = p1['teams']['player_1']['factories_to_place']
    log(f"player 0 has {num_plants0} plants to place")
    log(f"player 1 has {num_plants1} plants to place")
    rubble, ice, ore = p0['board']['rubble'], p0['board']['ice'], p0['board']['ore']
    assert (rubble == p1['board']['rubble']).all()
    assert (ice == p1['board']['ice']).all()
    assert (ore == p1['board']['ore']).all()
    log(f"Map has total {ice.sum()} ice, {ore.sum()} ore and {rubble.sum()} rubble")
    log(f"first col totals {ice[:, 0].sum()} ice, {ore[:, 0].sum()} ore and {rubble[:, 0].sum()} rubble")
    log(f"first row totals {ice[0, :].sum()} ice, {ore[0, :].sum()} ore and {rubble[0, :].sum()} rubble")

In [3]:
env = LuxAI_S2()

In [4]:
a1 = ControlledAgent('player_0', env.env_cfg, True, threshold=15, radius=130)
a2 = IdleAgent('player_1', env.env_cfg, True, threshold=15, radius=130)

In [5]:
agents = {a.player: a for a in [a1, a2]}
agents

{'player_0': <agents_experiments.ControlledAgent at 0x7f69fc5d1150>,
 'player_1': <agents_experiments.IdleAgent at 0x7f69fc5d1c90>}

In [6]:
game_step = step_interact(env, agents, 15, 5)

In [7]:
obs, rewards, dones, infos = [], [], [], []

In [8]:
known_step = -1

In [9]:
known_step += 1
o1, o2, o3, o4 = next(game_step)
obs.append(o1); rewards.append(o2); dones.append(o3); infos.append(o4)
diagnose(obs, rewards, dones, infos)

INFO:root:For both players real_env_steps=-4
INFO:root:player 0 has 2 plants to place
INFO:root:player 1 has 2 plants to place
INFO:root:Map has total 38 ice, 15 ore and 112145 rubble
INFO:root:first col totals 0 ice, 0 ore and 3877 rubble
INFO:root:first row totals 0 ice, 0 ore and 2787 rubble


In [10]:
known_step += 1
o1, o2, o3, o4 = next(game_step)
obs.append(o1); rewards.append(o2); dones.append(o3); infos.append(o4)
diagnose(obs, rewards, dones, infos)

INFO:root:Generate Spawners: 0.18
ERROR:root:options={'threshold': 15, 'radius': 130}


KeyError: 'player_0'

In [ ]:
known_step += 1
o1, o2, o3, o4 = next(game_step)
obs.append(o1); rewards.append(o2); dones.append(o3); infos.append(o4)
diagnose(obs, rewards, dones, infos)